In [ ]:
!pip install -q sentence-transformers faiss-cpu pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.6/23.6 MB 46.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 329.5/329.5 kB 13.8 MB/s eta 0:00:00


In [ ]:
# ============================================
# 1. IMPORTS
# ============================================

from pathlib import Path
import numpy as np
import faiss
from sentence_transformers import SentenceTransformer
from pypdf import PdfReader


# ============================================
# 2. CONFIGURACIÓN
# ============================================
# Ruta del PDF subido
PDF_PATH = "/content/1.-Basta-ya-2021-baja.pdf"   # <-- usa tu ruta exacta

# Modelo encoder: BGE-M3 (multilingüe, ideal para RAG)
EMBEDDING_MODEL_NAME = "BAAI/bge-m3"

# Parámetros del chunking
CHUNK_SIZE = 1200       # tamaño por chunk en caracteres
CHUNK_OVERLAP = 200     # solapamiento entre chunks


# ============================================
# 3. Lectura y chunking del PDF + METADATA
# ============================================

def load_pdf_text(pdf_path: str):
    """Devuelve lista de (num_pagina, texto)."""
    reader = PdfReader(pdf_path)
    pages = []
    for i, page in enumerate(reader.pages):
        try:
            text = page.extract_text() or ""
        except Exception:
            text = ""
        pages.append((i + 1, text.strip()))
    return pages


def chunk_page_text(page_num, text, pdf_name):
    """Crea chunks con metadata completa."""
    chunks = []
    start = 0
    chunk_idx = 0

    while start < len(text):
        end = start + CHUNK_SIZE
        chunk_text = text[start:end].strip()
        if not chunk_text:
            break

        # Metadata
        obj = {
            "pdf_name": pdf_name,
            "page": page_num,
            "chunk_id": f"{pdf_name}_p{page_num}_c{chunk_idx}",
            "text": chunk_text,
            "char_start": start,
            "char_end": min(end, len(text)),
            "position": chunk_idx
        }

        chunks.append(obj)
        chunk_idx += 1

        # Solapamiento
        start = end - CHUNK_OVERLAP

    return chunks


def build_corpus_from_pdf(pdf_path: str):
    """Devuelve lista de todos los chunks del PDF."""
    pdf_name = Path(pdf_path).name
    pages = load_pdf_text(pdf_path)

    corpus = []
    for page, text in pages:
        if text:
            corpus.extend(chunk_page_text(page, text, pdf_name))

    return corpus


# ============================================
# 4. SEARCHER con BGE-M3 + FAISS
# ============================================

class PDFSemanticSearcher:
    def __init__(self, pdf_path: str):

        print("Cargando modelo BGE-M3...")
        self.model = SentenceTransformer(EMBEDDING_MODEL_NAME)

        print("Construyendo corpus de chunks...")
        self.corpus = build_corpus_from_pdf(pdf_path)
        texts = [c["text"] for c in self.corpus]
        print(f"Total chunks: {len(texts)}")

        print("Generando embeddings con BGE-M3...")
        self.embeddings = self.model.encode(
            texts,
            show_progress_bar=True,
            normalize_embeddings=True   # recomendado para BGE
        )
        self.embeddings = np.array(self.embeddings).astype("float32")

        dim = self.embeddings.shape[1]
        print(f"Creando índice FAISS (dim={dim})...")
        self.index = faiss.IndexFlatL2(dim)
        self.index.add(self.embeddings)

        print("Índice listo ✔️\n")

    def search(self, query: str, top_k: int = 5):
        """Devuelve los top-K chunks más similares a la pregunta."""

        # Embedding de la pregunta
        query_vec = self.model.encode(
            [query],
            normalize_embeddings=True
        ).astype("float32")

        distances, indices = self.index.search(query_vec, top_k)
        distances, indices = distances[0], indices[0]

        results = []
        for dist, idx in zip(distances, indices):
            c = self.corpus[int(idx)]
            results.append({
                "score": float(dist),
                "pdf": c["pdf_name"],
                "page": c["page"],
                "chunk_id": c["chunk_id"],
                "char_range": (c["char_start"], c["char_end"]),
                "position": c["position"],
                "text": c["text"]
            })

        return results


# ============================================
# 5. BUCLE INTERACTIVO DE PREGUNTAS
# ============================================

if __name__ == "__main__":
    searcher = PDFSemanticSearcher(PDF_PATH)

    print("Buscador cargado. Pregunta lo que quieras del PDF.\n")

    while True:
        query = input("Pregunta: ").strip()
        if query.lower() in ["salir", "exit", "quit"]:
            print("Chao 👋")
            break

        results = searcher.search(query, top_k=5)

        print("\n========= TOP K RESULTADOS =========\n")
        for i, r in enumerate(results, 1):
            print(f"[{i}] Score: {r['score']:.4f}")
            print(f"PDF: {r['pdf']}")
            print(f"Página: {r['page']} | Chunk: {r['chunk_id']}")
            print(f"Rango caracteres: {r['char_range']}")
            print(f"Posición en página: {r['position']}")
            print("-" * 60)

            preview = r["text"]
            if len(preview) > 700:
                preview = preview[:700] + "..."
            print(preview)
            print("\n" + "="*80 + "\n")

Cargando modelo BGE-M3...
Construyendo corpus de chunks...
Total chunks: 1683
Generando embeddings con BGE-M3...


Batches:   0%|          | 0/53 [00:00<?, ?it/s]

Creando índice FAISS (dim=1024)...
Índice listo ✔️

Buscador cargado. Pregunta lo que quieras del PDF.

Pregunta: Cuales fueron los daños y las modalidades de violencia?

========= TOP K RESULTADOS =========

[1] Score: 0.7841
PDF: 1.-Basta-ya-2021-baja.pdf
Página: 259 | Chunk: 1.-Basta-ya-2021-baja.pdf_p259_c2
Rango caracteres: (2000, 3200)
Posición en página: 2
------------------------------------------------------------
do situaciones de 
horror extremo en condiciones de enorme indefensión y humillación. 
Sus victimarios fueron arbitrarios y no conocieron límites. Los testimo-
nios escuchados por el 
gmh ilustran la crueldad con la que actuaron los 
grupos armados y la sevicia con que cometieron los actos, así como su 
clara intención de sembrar el terror, instaurar el miedo, subyugar a la po-
blación y controlar los territorios. A los propósitos y cálculos estratégicos 
de las organizaciones armadas se sumaron prácticas de horror atroces e 
inimaginables que respondieron a retaliac

In [ ]:
# ============================================
# 5. BUCLE INTERACTIVO DE PREGUNTAS
# ============================================

if __name__ == "__main__":
    searcher = PDFSemanticSearcher(PDF_PATH)

    print("Buscador cargado. Pregunta lo que quieras del PDF.\n")

    while True:
        query = input("Pregunta: ").strip()
        if query.lower() in ["salir", "exit", "quit"]:
            print("Chao 👋")
            break

        results = searcher.search(query, top_k=5)

        print("\n========= TOP K RESULTADOS =========\n")
        for i, r in enumerate(results, 1):
            print(f"[{i}] Score: {r['score']:.4f}")
            print(f"PDF: {r['pdf']}")
            print(f"Página: {r['page']} | Chunk: {r['chunk_id']}")
            print(f"Rango caracteres: {r['char_range']}")
            print(f"Posición en página: {r['position']}")
            print("-" * 60)

            preview = r["text"]
            if len(preview) > 700:
                preview = preview[:700] + "..."
            print(preview)
            print("\n" + "="*80 + "\n")

Cargando modelo BGE-M3...
Construyendo corpus de chunks...
Total chunks: 1683
Generando embeddings con BGE-M3...


Batches:   0%|          | 0/53 [00:00<?, ?it/s]

Creando índice FAISS (dim=1024)...
Índice listo ✔️

Buscador cargado. Pregunta lo que quieras del PDF.

Pregunta: Cuales son son las modalidades de violencia?

========= TOP K RESULTADOS =========

[1] Score: 0.7235
PDF: 1.-Basta-ya-2021-baja.pdf
Página: 288 | Chunk: 1.-Basta-ya-2021-baja.pdf_p288_c3
Rango caracteres: (3000, 4200)
Posición en página: 3
------------------------------------------------------------
escritos previamente.
4.2.1. Las masacres: terror y devastación
Las masacres son una modalidad de violencia que combina experiencias 
del horror con graves y complejos impactos sobre sus víctimas. Como se 
pudo observar en el primer capítulo de este libro, las masacres fueron 
una práctica de violencia continua en el desarrollo del conflicto armado. 
Las masacres son una modalidad que los actores armados privilegian 
por su capacidad de instalar el terror y despoblar territorios. Los actos 
de barbarie que las caracterizan, y que fueron ampliamente descritos 
en el capítulo pri

In [ ]:
from pypdf import PdfReader
from pypdf.errors import PdfReadError

def check_pdf_integrity(pdf_path):
    try:
        reader = PdfReader(pdf_path)
        # Attempt to access a property to force parsing, like number of pages
        num_pages = len(reader.pages)
        print(f"PDF integrity check passed: Successfully read {num_pages} pages from '{pdf_path}'.")
        return True
    except PdfReadError as e:
        print(f"PDF integrity check failed for '{pdf_path}': {e}")
        print("This often indicates a corrupted, malformed, or encrypted PDF. Please ensure the file is valid and not password-protected.")
        return False
    except Exception as e:
        print(f"An unexpected error occurred while checking '{pdf_path}': {e}")
        return False

# Run the integrity check
check_pdf_integrity(PDF_PATH)

PDF integrity check passed: Successfully read 432 pages from '/content/1.-Basta-ya-2021-baja.pdf'.


True